# [Deep Learning](https://www.cc.gatech.edu/~hays/compvision/proj6/): Sketch_Data extra credit.

## Setup

In [1]:
%matplotlib notebook
%load_ext autoreload
%autoreload 2
import cv2
import numpy as np
import random
import torch.nn as nn
import torch.optim as optim
import os.path as osp
import matplotlib.pyplot as plt
from utils import *
import student_code as sc
from torchvision.models import alexnet
from torchsummary import summary

data_path = osp.join('../data', 'human_sketch')
num_classes = 250

# If you have a good Nvidia GPU with an appropriate environment, 
# try setting the use_GPU flag to True (the environment provided does
# not support GPUs and we will not provide any support for GPU
# computation in this project). Please note that 
# we will evaluate your implementations only using CPU mode so even if
# you use a GPU, make sure your code runs in the CPU mode with the
# environment we provided. 
use_GPU = True
if use_GPU:
    from utils_gpu import *

To train a network in PyTorch, we need 4 components:
1. **Dataset** - an object which can load the data and labels given an index.
2. **Model** - an object that contains the network architecture definition.
3. **Loss function** - a function that measures how far the network output is from the ground truth label.
4. **Optimizer** - an object that optimizes the network parameters to reduce the loss value.

This project has two main parts. In Part 1, you will train a deep network from scratch. In Part 2, you will "fine-tune" a trained network. 

## Part 0. Warm up! Training a Deep Network from Scratch

In [2]:
# Fix random seeds so that results will be reproducible
set_seed(0, use_GPU)

You do not need to code anything for this part. You will simply run the code we provided, but we want you to report the result you got. This section will also familiarize you with the steps of training a deep network from scratch. 

In [3]:
# Training parameters.
input_size = (64, 64)
RGB = False  
base_lr = 1e-2  # may try a smaller lr if not using batch norm
weight_decay = 5e-4
momentum = 0.9

We will first create our datasets, by calling the create_datasets function from student_code. This function returns a separate dataset loader for each split of the dataset (training and testing/validation). Each dataloader is used to load the datasets after appling some pre-processing transforms. In Part 1, you will be asked to add a few more pre-processing transforms to the dataloaders by modifying this function.

In [4]:
# Create the training and testing datasets.
train_dataset, test_dataset = sc.create_datasets(data_path=data_path, input_size=input_size, rgb=RGB)
assert test_dataset.classes == train_dataset.classes

Computing pixel mean and stdev...
Batch 0 / 200
Batch 20 / 200
Batch 40 / 200
Batch 60 / 200
Batch 80 / 200
Batch 100 / 200
Batch 120 / 200
Batch 140 / 200
Batch 160 / 200
Batch 180 / 200
Done, mean = 
[0.982125]
std = 
[0.05606342]
Computing pixel mean and stdev...
Batch 0 / 200
Batch 20 / 200
Batch 40 / 200
Batch 60 / 200
Batch 80 / 200
Batch 100 / 200
Batch 120 / 200
Batch 140 / 200
Batch 160 / 200
Batch 180 / 200
Done, mean = 
[0.98213223]
std = 
[0.05596984]


Now we will create our network model using the SketchNet class from student_code. The implementation provided in the SketchNet class gives you a basic network. In Part 1, you will be asked to add a few more layers to this network. 

In [5]:
# Create the network model.
model = sc.SketchNet(num_classes=num_classes, rgb=RGB, verbose=False)
if use_GPU:
    model = model.cuda()
print(model)
# summary(model, (1, 64, 64))

SketchNet(
  (features): Sequential(
    (0): Conv2d(1, 20, kernel_size=(9, 9), stride=(1, 1), bias=False)
    (1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (2): ReLU()
    (3): BatchNorm2d(20, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (4): Conv2d(20, 30, kernel_size=(5, 5), stride=(1, 1), bias=False)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): ReLU()
    (7): BatchNorm2d(30, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): Conv2d(30, 50, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (9): ReLU()
    (10): BatchNorm2d(50, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): Dropout(p=0.5)
  )
  (classifier): Conv2d(50, 250, kernel_size=(8, 8), stride=(1, 1))
)


Next we will create the loss function and the optimizer. 

In [6]:
# Create the loss function.
# see http://pytorch.org/docs/0.3.0/nn.html#loss-functions for a list of available loss functions
loss_function = nn.CrossEntropyLoss()

In [7]:
# Create the optimizer and a learning rate scheduler
optimizer = optim.SGD(params=model.parameters(), lr=base_lr, weight_decay=weight_decay, momentum=momentum)
# Currently a simple step scheduler.
# See http://pytorch.org/docs/0.3.0/optim.html#how-to-adjust-learning-rate for various LR schedulers
# and how to use them
lr_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=60, gamma=0.1)

Finally we are ready to train our network! We will start a local server to see the training progress of our network. Open a new terminal and activate the environment for this project. Then run the following command: **python -m visdom.server**. This will start a local server. The terminal output should give out a link like: "http://localhost:8097". Open this link in your browser. After you run the following block, visit this link again, and you will be able to see graphs showing the progress of your training! If you do not see any graphs, select Part 1 on the top left bar where is says Environment (only select Part 1, do not check main or Part 2).

In [10]:
# train the network!
params = {'n_epochs': 5, 'batch_size': 100, 'experiment': 'part3'}
trainer = Trainer(train_dataset, test_dataset, model, loss_function, optimizer, lr_scheduler, params)
best_prec1 = trainer.train_val()
print('Best top-1 Accuracy = {:4.3f}'.format(best_prec1))

---------------------------------------
Experiment: part3
experiment: part3
n_epochs: 5
print_freq: 100
resume_optim: True
checkpoint_file: None
shuffle: True
num_workers: 4
val_freq: 1
batch_size: 100
do_val: True
---------------------------------------
part3 Epoch 0 / 5
train part3: batch 0/99, loss 1.786, top-1 accuracy 58.000, top-5 accuracy 79.000
train part3: loss 1.539305
val part3: batch 0/99, loss 3.027, top-1 accuracy 45.000, top-5 accuracy 60.000
val part3: loss 2.470322
Checkpoint saved
BEST TOP1 ACCURACY SO FAR
part3 Epoch 1 / 5
train part3: batch 0/99, loss 1.494, top-1 accuracy 66.000, top-5 accuracy 87.000
train part3: loss 1.463846
val part3: batch 0/99, loss 3.080, top-1 accuracy 42.000, top-5 accuracy 61.000
val part3: loss 2.473781
Checkpoint saved
BEST TOP1 ACCURACY SO FAR
part3 Epoch 2 / 5
train part3: batch 0/99, loss 1.685, top-1 accuracy 57.000, top-5 accuracy 85.000
train part3: loss 1.379656
val part3: batch 0/99, loss 3.111, top-1 accuracy 40.000, top-5 accu

In [11]:
# summary(model, (1, 64, 64))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 20, 56, 56]           1,620
         MaxPool2d-2           [-1, 20, 27, 27]               0
              ReLU-3           [-1, 20, 27, 27]               0
       BatchNorm2d-4           [-1, 20, 27, 27]              40
            Conv2d-5           [-1, 30, 23, 23]          15,000
         MaxPool2d-6           [-1, 30, 11, 11]               0
              ReLU-7           [-1, 30, 11, 11]               0
       BatchNorm2d-8           [-1, 30, 11, 11]              60
            Conv2d-9             [-1, 50, 8, 8]          24,000
             ReLU-10             [-1, 50, 8, 8]               0
      BatchNorm2d-11             [-1, 50, 8, 8]             100
          Dropout-12             [-1, 50, 8, 8]               0
           Conv2d-13            [-1, 250, 1, 1]         800,250
Total params: 841,070
Trainable params:

## Part 3.2: Using AlexNet to classify sketch data

In [13]:
# Fix random seeds so that results will be reproducible
set_seed(0, use_GPU)

# training parameters
input_size = (224, 224)
RGB = True
base_lr = 1e-3
weight_decay = 5e-4
momentum = 0.9
backprop_depth = 3


# Create the training and testing datasets.
train_dataset, test_dataset = sc.create_datasets(data_path=data_path, input_size=input_size, rgb=RGB)
assert test_dataset.classes == train_dataset.classes

# Create the network model.
model = alexnet(pretrained=True)
print(model)

model = sc.create_part2_model(model, num_classes)
if use_GPU:
    model = model.cuda()
print(model)
# summary(model, (3, 224, 224))


# Set up the trainer. You can modify custom_part2_trainer in
# student_copy.py if you want to try different learning settings.
custom_part2_trainer = sc.custom_part2_trainer(model)

if custom_part2_trainer is None:
    # Create the loss function
    # see http://pytorch.org/docs/0.3.0/nn.html#loss-functions for a list of available loss functions
    loss_function = nn.CrossEntropyLoss()

    # Since we do not want to optimize the whole network, we must extract a list of parameters of interest that will be
    # optimized by the optimizer.
    params_to_optimize = []

    # List of modules in the network
    mods = list(model.features.children()) + list(model.classifier.children())

    # Extract parameters from the last `backprop_depth` modules in the network and collect them in
    # the params_to_optimize list.
    for m in mods[::-1][:backprop_depth]:
        params_to_optimize.extend(list(m.parameters()))

    # Construct the optimizer    
    optimizer = optim.SGD(params=params_to_optimize, lr=base_lr, weight_decay=weight_decay, momentum=momentum)

    # Create a scheduler, currently a simple step scheduler, but you can get creative.
    # See http://pytorch.org/docs/0.3.0/optim.html#how-to-adjust-learning-rate for various LR schedulers
    # and how to use them
    lr_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)
    
    params = {'n_epochs': 4, 'batch_size': 10, 'experiment': 'part4'} 
    
else:
    if 'loss_function' in custom_part2_trainer:
        loss_function = custom_part2_trainer['loss_function']
    if 'optimizer' in custom_part2_trainer:
        optimizer = custom_part2_trainer['optimizer']
    if 'lr_scheduler' in custom_part2_trainer:
        lr_scheduler = custom_part2_trainer['lr_scheduler']
    if 'params' in custom_part2_trainer:
        params = custom_part2_trainer['params']
        
        
# Train the network!
trainer = Trainer(train_dataset, test_dataset, model, loss_function, optimizer, lr_scheduler, params)
best_prec1 = trainer.train_val()
print('Best top-1 Accuracy = {:4.3f}'.format(best_prec1))

Computing pixel mean and stdev...
Batch 0 / 200
Batch 20 / 200
Batch 40 / 200
Batch 60 / 200
Batch 80 / 200
Batch 100 / 200
Batch 120 / 200
Batch 140 / 200
Batch 160 / 200
Batch 180 / 200
Done, mean = 
[0.98226709 0.98226709 0.98226709]
std = 
[0.09226456 0.09226456 0.09226456]
Computing pixel mean and stdev...
Batch 0 / 200
Batch 20 / 200
Batch 40 / 200
Batch 60 / 200
Batch 80 / 200
Batch 100 / 200
Batch 120 / 200
Batch 140 / 200
Batch 160 / 200
Batch 180 / 200
Done, mean = 
[0.98227521 0.98227521 0.98227521]
std = 
[0.09220306 0.09220306 0.09220306]
AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3,

val part4: batch 500/999, loss 4.581, top-1 accuracy 0.000, top-5 accuracy 10.000
val part4: batch 600/999, loss 3.306, top-1 accuracy 20.000, top-5 accuracy 60.000
val part4: batch 700/999, loss 4.362, top-1 accuracy 0.000, top-5 accuracy 0.000
val part4: batch 800/999, loss 3.129, top-1 accuracy 50.000, top-5 accuracy 60.000
val part4: batch 900/999, loss 4.864, top-1 accuracy 0.000, top-5 accuracy 30.000
val part4: loss 3.229136
Checkpoint saved
BEST TOP1 ACCURACY SO FAR
part4 Epoch 2 / 4
train part4: batch 0/999, loss 2.120, top-1 accuracy 40.000, top-5 accuracy 70.000
train part4: batch 100/999, loss 3.571, top-1 accuracy 20.000, top-5 accuracy 30.000
train part4: batch 200/999, loss 3.305, top-1 accuracy 30.000, top-5 accuracy 60.000
train part4: batch 300/999, loss 3.037, top-1 accuracy 20.000, top-5 accuracy 50.000
train part4: batch 400/999, loss 2.354, top-1 accuracy 50.000, top-5 accuracy 60.000
train part4: batch 500/999, loss 4.020, top-1 accuracy 10.000, top-5 accuracy 40